In [1]:
import numpy as np
import pandas as pd

In [153]:
text = open('chatbotmodels/phoenixaa.txt').read().lower()
text = text.split('\n')

In [154]:
cleaned_text = []

for line in text:
    if line != '' and line[0] not in ['-','+','*']:
            cleaned_text.append(line)

ix = 0
new_cleaned_text = []

for line in cleaned_text:
    if line[-1] == ":":
        new_cleaned_text.append(line)
        fixed_line = []
        for next_line in cleaned_text[ix+1:]:
            if next_line[-1] == ":":
                break
            else:
                fixed_line.append(next_line)
        new_cleaned_text.append(' '.join(fixed_line))
    ix += 1

In [155]:
text = new_cleaned_text

def get_dialouge(character):
    char_dialouge = []
    prompt = []
    ix = 0
    for line in text:
        if line == character+':':
            char_dialouge.append(text[ix+1])
            prompt.append(text[ix-1])
        ix += 1
    return prompt, char_dialouge

In [156]:
prompt, reply = get_dialouge('maya')
aadf = pd.DataFrame({'prompt':prompt, 'reply':reply})

In [157]:
text = open('phoenix.txt').read()
test = text.split('\n')

In [158]:
# Clean text
cleaned_text = []
ix = 0

for i in test:
    if i != '':
        if i[0] == '-' and i[1] not in ['-','=']:
            fix_ix = ix + 1
            fixed_line = []
            cleaned_text.append(i)
            for line in test[ix+1:]:
                if line in ['', ' ', '  ', '   ', '    ']:
                    break
                else:
                    fixed_line.append(line)
            fixed_line = ' '.join(' '.join(fixed_line).split())
            cleaned_text.append(fixed_line)
    ix += 1
cleaned_text = '\n'.join(cleaned_text)

In [159]:
list_text = cleaned_text.split('\n')

def get_dialouge(character):
    char_dialouge = []
    prompt = []
    ix = 0
    for line in list_text:
        if line == '-'+character and list_text[ix+1] != '...':
            char_dialouge.append(list_text[ix+1].lower())
            prompt.append(list_text[ix-1].lower())
        ix += 1
    return prompt, char_dialouge

In [160]:
prompt, reply = get_dialouge('Maya')
aaadf = pd.DataFrame({'prompt':prompt, 'reply':reply})

In [161]:
df = pd.concat([aadf,aaadf], axis=0)

In [162]:
prompt = list(df['prompt'])
reply = list(df['reply'])

In [164]:
prompt[143]

"(i hope maya's ok. i should hurry and get to the detention center, asap!) june 20, 10:34 am detention center visitor's room"

In [151]:
reply[332]

"that... that's terrible... why? why would my aunt...? everything is going just as i predicted, mr. phoenix wright."

In [163]:
max_len = 0
max_position = 0
count = 0
for i in prompt:
    if len(i) > max_len:
        max_len = len(i)
        max_position = count
    count += 1
print("Prompt max line length: "+str(max_len))
print(max_position)

max_position = 0
count = 0
max_len = 0
for i in reply:
    if len(i) > max_len:
        max_len = len(i)
        max_position = count
    count += 1
print("Reply max line length: "+str(max_len))
print(max_position)

Prompt max line length: 123
143
Reply max line length: 114
332


In [165]:
max_seq_len = 40

In [166]:
y = []
for line in reply:
    y.append(line[:max_seq_len])

In [167]:
def start_end(line):
    line = '\t'+line+'\n'
    return line

In [168]:
Y = [start_end(line) for line in y]

In [18]:
# def pad(line):
#     if len(line) < max_seq_len:
#         line += ' '*(max_seq_len-len(line))
#     return line

In [26]:
# x_padded = [pad(line) for line in prompt]
# y_padded = [pad(line) for line in reply]

In [169]:
prompt_chars = set(' '.join(prompt))
reply_chars = set(' '.join(Y))
# all_chars = []

# for i in prompt_chars:
#     if i not in all_chars:
#         all_chars.append(i)
# for i in reply_chars:
#     if i not in all_chars:
#         all_chars.append(i)
# all_chars = sorted(list(set(all_chars)))

In [172]:
prompt_char_to_ix = dict((c, i) for i, c in enumerate(prompt_chars))
prompt_ix_to_char = dict((i, c) for i, c in enumerate(prompt_chars))
reply_char_to_ix = dict((c, i) for i, c in enumerate(reply_chars))
reply_ix_to_char = dict((i, c) for i, c in enumerate(reply_chars))

In [171]:
# char_to_ix = dict((c, i) for i, c in enumerate(all_chars))
# ix_to_char = dict((i, c) for i, c in enumerate(all_chars))

NameError: name 'all_chars' is not defined

In [173]:
prompt_feature_size = len(prompt_chars)
reply_feature_size = len(reply_chars)

In [174]:
X = []

for i in range(len(prompt)):
    x_seq = prompt[i][:max_seq_len]
    one_hot = np.zeros((max_seq_len, prompt_feature_size))
    x_seq = [prompt_char_to_ix[c] for c in x_seq]
    for j in range(len(x_seq)):
        one_hot[j][x_seq[j]] = 1
    X.append(one_hot)
X = np.asarray(X)

print("INPUT DATA SHAPE: "+str(X.shape))

INPUT DATA SHAPE: (2709, 40, 53)


In [175]:
y = []

for i in range(len(Y)):
    y_seq= Y[i][:max_seq_len]
    one_hot = np.zeros((max_seq_len, reply_feature_size))
    y_seq = [reply_char_to_ix[c] for c in y_seq]
    for j in range(len(y_seq)):
        one_hot[j][y_seq[j]] = 1
    y.append(one_hot)
y = np.asarray(y)

print("OUTPUT DATA SHAPE: "+str(y.shape))

OUTPUT DATA SHAPE: (2709, 40, 54)


In [176]:
y_input = y[:,:39,:]
print(y_input.shape)
y_output = y[:,1:,:]
print(y_output.shape)

(2709, 39, 54)
(2709, 39, 54)


In [177]:
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed, Activation, RepeatVector, Input
from keras.callbacks import ModelCheckpoint, Callback

C:\Users\Gary\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [178]:
encoder_inputs = Input(shape=(None, prompt_feature_size))
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, reply_feature_size))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(reply_feature_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 53)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 54)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 93184       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  93696       input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [230]:
model.fit([X, y_input], y_output,
          epochs=30)

Epoch 1/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1766A: 0s - loss: 0
Epoch 2/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1733
Epoch 3/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1745
Epoch 4/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1732
Epoch 5/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1754
Epoch 6/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1693
Epoch 7/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1718
Epoch 8/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1727
Epoch 9/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1717
Epoch 10/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1685
Epoch 11/30
2709/2709 [==============================] - 5s 2ms/step - loss: 0.1681
Epoch 12/30
2709/2709 [==============================] - 5s 2ms/step -

In [180]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [181]:
def process_question(question):
    question_vec = [prompt_char_to_ix[c] for c in question]
    
    one_hot = np.zeros((max_seq_len, prompt_feature_size))
    
    for j in range(len(question_vec)):
        one_hot[j][question_vec[j]] = 1
    
    return one_hot.reshape(-1,max_seq_len,prompt_feature_size)

In [182]:
def gen_reply(question):
    question = process_question(question)
    
    states_value = encoder_model.predict(question)
    
    target_seq = np.zeros((1, 1, reply_feature_size))
    target_seq[0, 0, reply_char_to_ix['\t']] = 1.
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reply_ix_to_char[sampled_token_index]
        decoded_sentence += sampled_char
        
        if (sampled_char == '\n' or len(decoded_sentence) > max_seq_len):
            stop_condition = True

        target_seq = np.zeros((1, 1, reply_feature_size))
        target_seq[0, 0, sampled_token_index] = 1.
        
        states_value = [h, c]
        
    return decoded_sentence

In [253]:
rand = prompt[np.random.randint(len(prompt))][:max_seq_len]
print(rand)
print('\n')
print(gen_reply(rand))

i kind of wish she'd said a little more.


it's hard. i think she's really will.



In [301]:
print(gen_reply(new_cleaned_text[np.random.randint(len(new_cleaned_text))][:40]))

hey casey...



In [263]:
model.save_weights('chatbotweights.h5')

In [262]:
model.save('chatbot.h5')
encoder_model.save('chatbotenc.h5')
decoder_model.save('chatbotdec.h5')

C:\Users\Gary\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\engine\topology.py:2364: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '
C:\Users\Gary\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\engine\topology.py:2364: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_6:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'input_7:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [31]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_seq_len, feature_size)))
model.add(RepeatVector(max_seq_len))
model.add(LSTM(128, return_sequences=True))
model.add(TimeDistributed(Dense(feature_size)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               92672     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 30, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 128)           131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 52)            6708      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 52)            0         
Total params: 230,964
Trainable params: 230,964
Non-trainable params: 0
_________________________________________________________________


In [75]:
model.fit(X, y, epochs=500)

Epoch 1/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8247
Epoch 2/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8229
Epoch 3/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8253A: 1s - 
Epoch 4/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8228A: 0s - loss: 0.8
Epoch 5/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8300
Epoch 6/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8309
Epoch 7/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8140
Epoch 8/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8252
Epoch 9/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8170
Epoch 10/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8137
Epoch 11/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.8145A: 1s
Epoch 12/500
2116/2116 [====================

2116/2116 [==============================] - 3s 1ms/step - loss: 0.6279
Epoch 190/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6291
Epoch 191/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6273
Epoch 192/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6210
Epoch 193/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6272
Epoch 194/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6230
Epoch 195/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6148
Epoch 196/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6235
Epoch 197/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6200
Epoch 198/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6188
Epoch 199/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.6088
Epoch 200/500
2116/2116 [==============================] - 3s 1ms/st

Epoch 282/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5788
Epoch 283/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5786
Epoch 284/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5721
Epoch 285/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5594
Epoch 286/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5638
Epoch 287/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5733
Epoch 288/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5570
Epoch 289/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5689
Epoch 290/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5702
Epoch 291/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5525
Epoch 292/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.5678A: 1s - 
Epoch 293/500
2116/2116 [=====================

2116/2116 [==============================] - 3s 1ms/step - loss: 0.4917
Epoch 469/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4892
Epoch 470/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4812
Epoch 471/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4881
Epoch 472/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4875
Epoch 473/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4808
Epoch 474/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4916
Epoch 475/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4933
Epoch 476/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4822A: 0s - lo
Epoch 477/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4879
Epoch 478/500
2116/2116 [==============================] - 3s 1ms/step - loss: 0.4997
Epoch 479/500
2116/2116 [==============================] -

In [114]:
def process_question(question):
    question = pad(question)
    question_vec = [char_to_ix[c] for c in question]
    
    one_hot = np.zeros((max_seq_len, feature_size))
    
    for j in range(len(question_vec)):
        one_hot[j][question_vec[j]] = 1
    
    return one_hot.reshape(-1,max_seq_len,feature_size)

In [141]:
rand = prompt[np.random.randint(len(prompt))][:max_seq_len]
print(rand)
print('\n')
for i in np.argmax(model.predict(process_question(rand))[0],1):
    print(ix_to_char[i], end='')

(this girl is more troubled th


look, a ladder!               

In [150]:
for i in np.argmax(model.predict(process_question('what do you want to eat?'))[0],1):
    print(ix_to_char[i], end='')

yoof!iill!yllellyy aya.       

In [57]:
ix_to_char[np.argmax(model.predict(process_question("huh?"))[0][3])]

' '